[View in Colaboratory](https://colab.research.google.com/github/anu0508/new-inning/blob/master/Project3ACDgild.ipynb)

In [0]:
import numpy as np
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [0]:
test_set =pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', skiprows = 1, header = None)


In [0]:

train_set =pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)

In [0]:
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status','occupation','relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week','native_country', 'wage_class']


In [0]:

train_set.columns = col_labels
test_set.columns = col_labels

In [6]:
train_set['workclass'].value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64

In [7]:
train_set['education'].value_counts()

 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64

In [8]:
train_set['marital_status'].value_counts()

 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital_status, dtype: int64

In [9]:
train_set.shape #

(32561, 15)

In [10]:
test_set.shape #

(16281, 15)

In [0]:
train_set['data_id']= 1 #before conacating data on axis=0 we are creating a column with 1 and 0. train set id will 1 and test 0. so that later on we can remove 

In [0]:
test_set['data_id']= 0

In [13]:
train_set.shape

(32561, 16)

In [0]:
#combine data
total_set=train_set.append(test_set)

In [15]:

total_set['wage_class'].value_counts()

 <=50K     24720
 <=50K.    12435
 >50K       7841
 >50K.      3846
Name: wage_class, dtype: int64

In [0]:

import numpy as np
df1= total_set.replace({' ?':np.nan})

In [17]:
df1.isnull().sum()

age                  0
workclass         2799
fnlwgt               0
education            0
education_num        0
marital_status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     857
wage_class           0
data_id              0
dtype: int64

In [0]:
#fillna with unknown
df1.fillna('unknown', inplace=True)

In [19]:
df1.isnull().sum() #no of nan to check if nan are replaced with unknown

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
wage_class        0
data_id           0
dtype: int64

In [20]:
df1['workclass'].value_counts()

 Private             33906
 Self-emp-not-inc     3862
 Local-gov            3136
unknown               2799
 State-gov            1981
 Self-emp-inc         1695
 Federal-gov          1432
 Without-pay            21
 Never-worked           10
Name: workclass, dtype: int64

In [21]:
df1['wage_class'].unique()

array([' <=50K', ' >50K', ' <=50K.', ' >50K.'], dtype=object)

In [0]:
df1['Target']=0
df1.loc[df1['wage_class']==' >50K','Target']=1
df1.loc[df1['wage_class']==' >50K.','Target']=1

In [23]:
df1.Target.value_counts()

0    37155
1    11687
Name: Target, dtype: int64

In [0]:
### copy of the df1

In [0]:
df2= df1.copy()

In [26]:
df2.shape

(48842, 17)

In [27]:
df2.Target.value_counts()

0    37155
1    11687
Name: Target, dtype: int64

In [0]:

df2.loc[df2['workclass']==' Without -pay','work_class']='not_working'
df2.loc[df2['workclass']==' Never-worked','work_class']='not_working'

df2.loc[df2['workclass']==' Local-gov','work_class']='Gov_job'
df2.loc[df2['workclass']==' State-gov','work_class']='Gov_job'
df2.loc[df2['workclass']==' Federal-gov','work_class']='Gov_job'

df2.loc[df2['workclass']==' Self-emp-inc','work_class']='self_emp'
df2.loc[df2['workclass']==' Self-emp-not-inc','work_class']='self_emp'

df2.loc[df2['workclass']==' Private','work_class']='Private'

df2.loc[df2['workclass']=='unknown','work_class']='unknown'



In [29]:
df2.work_class.value_counts()

Private        33906
Gov_job         6549
self_emp        5557
unknown         2799
not_working       10
Name: work_class, dtype: int64

In [30]:
df2.marital_status.value_counts()

 Married-civ-spouse       22379
 Never-married            16117
 Divorced                  6633
 Separated                 1530
 Widowed                   1518
 Married-spouse-absent      628
 Married-AF-spouse           37
Name: marital_status, dtype: int64

In [0]:
df2['Marital_status']='Married'

df2.loc[df2['marital_status']==' Never-married','Marital_status']='Single'

df2.loc[df2['marital_status']==' Divorced','Marital_status']='Separated'
df2.loc[df2['marital_status']==' Separated','Marital_status']='Separated'
df2.loc[df2['marital_status']==' Widowed','Marital_status']='Separated'
df2.loc[df2['marital_status']==' Married-spouse-absent','Marital_status']='Separated'



In [32]:
df2['Marital_status'].value_counts()

Married      22416
Single       16117
Separated    10309
Name: Marital_status, dtype: int64

In [0]:
#Education

In [34]:
df2['education'].value_counts()

 HS-grad         15784
 Some-college    10878
 Bachelors        8025
 Masters          2657
 Assoc-voc        2061
 11th             1812
 Assoc-acdm       1601
 10th             1389
 7th-8th           955
 Prof-school       834
 9th               756
 12th              657
 Doctorate         594
 5th-6th           509
 1st-4th           247
 Preschool          83
Name: education, dtype: int64

In [0]:
df2['Eductaion']='<HS grad'

In [0]:
edu_cat=[' Some-college',' Bachelors']

In [0]:

for cat in edu_cat:
    df2.loc[df2['education']==cat,'Eductaion']='Bachelors'
    

In [0]:

df2.loc[df2['education']==' Doctorate','Eductaion']='>=Masters'
df2.loc[df2['education']==' Masters','Eductaion']='>=Masters'

In [39]:
df2.Eductaion.value_counts()

<HS grad     26688
Bachelors    18903
>=Masters     3251
Name: Eductaion, dtype: int64

In [0]:
### Occupation

In [41]:
df2['occupation'].value_counts()

 Prof-specialty       6172
 Craft-repair         6112
 Exec-managerial      6086
 Adm-clerical         5611
 Sales                5504
 Other-service        4923
 Machine-op-inspct    3022
unknown               2809
 Transport-moving     2355
 Handlers-cleaners    2072
 Farming-fishing      1490
 Tech-support         1446
 Protective-serv       983
 Priv-house-serv       242
 Armed-Forces           15
Name: occupation, dtype: int64

In [0]:
executives=[' Exec-managerial',' Prof-specialty']
Technicians=[' Craft-repair',' Machine-op-inspct',' Other-service',' Armed-Forces']
Workers=[' Transport-moving',' Handlers-cleaners',' Farming-fishing',' Protective-serv', ' Priv-house-serv']
sales=[' Adm-clerical',' Sales',' Tech-support']
unknown=['unknown']


In [0]:
for e in Technicians:
    df2.loc[df2['occupation']==e,'Occupation']='Technicians'
for e in Workers:
    df2.loc[df2['occupation']==e,'Occupation']='Workers'
for e in sales:
    df2.loc[df2['occupation']==e,'Occupation']='sales'
for e in unknown:
    df2.loc[df2['occupation']==e,'Occupation']='unknown'

In [44]:
df2['Occupation'].unique()

array(['sales', nan, 'Workers', 'Technicians', 'unknown'], dtype=object)

In [45]:
df2['Occupation'].value_counts()

Technicians    14072
sales          12561
Workers         7142
unknown         2809
Name: Occupation, dtype: int64

In [46]:
#Race
df2['Race']='White'
df2.loc[df2['race']==' Black','Race']='Black'

df2.loc[df2['race']==' Asian-Pac-Islander','Race']='other'
df2.loc[df2['race']==' Amer-Indian-Eskimo','Race']='other'
df2.loc[df2['race']==' Other','Race']='other'

df2.Race.value_counts()

White    41762
Black     4685
other     2395
Name: Race, dtype: int64

In [47]:
#Relationship
df2.relationship.value_counts()
df2['Relationship']='spouse'
#spouse=[' Husband',' Wife']
relative=[' Not-in-family',' Other-relative']
unmarried=[' Own-child',' Unmarried']


for r in relative:
    df2.loc[df2['relationship']==r,'Relationship']='Relative'
for r in unmarried:
    df2.loc[df2['relationship']==r,'Relationship']='unmarried'

    
df2.Relationship.unique()
df2.Relationship.value_counts()


spouse       22047
Relative     14089
unmarried    12706
Name: Relationship, dtype: int64

In [0]:
### Native_country
df2['native_country'].unique()
Europe=[' England',' Germany',' Italy',' Poland',' Portugal',' France',' Yugoslavia',' Scotland',' Greece',' Ireland',' Hungary',' Holand-Netherlands']
Asia=[' India',' Iran',' Philippines',' Cambodia',' Thailand',' Taiwan',' China',' Japan',' Vietnam',' Hong',]
df2['Native_country']='America'

for e in Europe:
    df2.loc[df2['native_country']==e,'Native_country']='Europe'

for e in Asia:
    df2.loc[df2['native_country']==e,'Native_country']='Asia'    

df2.loc[df2['native_country']=='unknown','Native_country']='unknown'    


In [49]:
df2.Native_country.value_counts()


America    46247
Asia         958
unknown      857
Europe       780
Name: Native_country, dtype: int64

In [50]:
df2.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'wage_class', 'data_id', 'Target', 'work_class', 'Marital_status',
       'Eductaion', 'Occupation', 'Race', 'Relationship', 'Native_country'],
      dtype='object')

In [0]:
dropped=df2.drop(columns=['workclass','education', 'wage_class','marital_status','occupation','relationship','race','native_country'],axis=1,inplace=True)

In [52]:
df2.columns

Index(['age', 'fnlwgt', 'education_num', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'data_id', 'Target', 'work_class', 'Marital_status',
       'Eductaion', 'Occupation', 'Race', 'Relationship', 'Native_country'],
      dtype='object')

In [53]:
df2.head()

,age,fnlwgt,education_num,sex,capital_gain,capital_loss,hours_per_week,data_id,Target,work_class,Marital_status,Eductaion,Occupation,Race,Relationship,Native_country
0,39,77516,13,Male,2174,0,40,1,0,Gov_job,Single,Bachelors,sales,White,Relative,America
1,50,83311,13,Male,0,0,13,1,0,self_emp,Married,Bachelors,NaN,White,spouse,America
2,38,215646,9,Male,0,0,40,1,0,Private,Separated,<HS grad,Workers,White,Relative,America
3,53,234721,7,Male,0,0,40,1,0,Private,Married,<HS grad,Workers,Black,spouse,America
4,28,338409,13,Female,0,0,40,1,0,Private,Married,Bachelors,NaN,Black,spouse,America


In [0]:
#Label encoder for column sex


from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(df2['sex'])
df2['sex']=le.transform(df2['sex']) #tranforming train and test data
#test_data[col]=le.transform(test_data[col])

In [55]:
df2.columns

Index(['age', 'fnlwgt', 'education_num', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'data_id', 'Target', 'work_class', 'Marital_status',
       'Eductaion', 'Occupation', 'Race', 'Relationship', 'Native_country'],
      dtype='object')

In [0]:
cat_cols=['work_class','Marital_status','Eductaion','Occupation','Race','Relationship','Native_country']

In [0]:

dummies=pd.get_dummies(df2['Race'],prefix='Race')
df2 = df2.join(dummies)
#df2 = df2.drop(df2['Race'], axis = 1)

In [0]:
dummies=pd.get_dummies(df2['Native_country'],prefix='Country')
df2 = df2.join(dummies)

In [0]:
dummies=pd.get_dummies(df2['Relationship'],prefix='Relation')
df2 = df2.join(dummies)

In [0]:
dummies=pd.get_dummies(df2['work_class'],prefix='work')
df2 = df2.join(dummies)

In [0]:
dummies=pd.get_dummies(df2['Eductaion'],prefix='edu')
df2 = df2.join(dummies)

In [0]:
dummies=pd.get_dummies(df2['Marital_status'],prefix='status')
df2 = df2.join(dummies)

In [0]:
dummies=pd.get_dummies(df2['Occupation'],prefix='Occupation')
df2 = df2.join(dummies)